Creating an LSTM (Long Short-Term Memory) model for poem generation involves several steps. LSTM is a type of recurrent neural network that is particularly well-suited for sequence-to-sequence tasks like generating poems. Here's a step-by-step guide to building an LSTM model for poem creation:

1. **Data Collection**:
   Gather a large dataset of poems that will serve as the training data for your model. You can find poems from online sources, public domain poetry collections, or even create your own dataset.

2. **Data Preprocessing**:
   Prepare the raw text data for training. This involves tasks such as tokenization, lowercasing, removing punctuation, and creating numerical representations of the words. You can use libraries like NLTK or TensorFlow Tokenizer for this step.

3. **Create Sequences**:
   Convert the processed text into sequences of fixed length. For instance, if your LSTM model takes sequences of 50 words as input, divide the entire poem dataset into overlapping sequences of 50 words.

4. **Split Data into Train and Test Sets**:
   Divide the dataset into training and testing sets. The training set will be used to train the LSTM model, while the testing set will help evaluate its performance.

5. **Build LSTM Model**:
   Set up the architecture of your LSTM model. In Keras or TensorFlow, you can use the `LSTM` layer along with `Embedding` and `Dense` layers to create the model.

6. **Compile Model**:
   Specify the loss function and optimization algorithm for the LSTM model using the `compile` method. Since poem generation is a language generation task, you can use categorical cross-entropy as the loss function.

7. **Train Model**:
   Train the LSTM model on the prepared training dataset. Adjust the number of epochs and batch size according to your dataset size and computing resources. You might need to experiment with hyperparameters to achieve good results.

8. **Generate Poems**:
   After training the model, you can use it to generate poems. Start with a seed sequence, and use the trained model to predict the next word in the sequence. Then, use the predicted word to update the sequence, and repeat the process to generate the desired length of the poem.

9. **Temperature Sampling**:
   To control the creativity of the generated poems, you can introduce a temperature parameter in the sampling process. Higher values of temperature will result in more random and diverse output, while lower values will produce more focused and deterministic results.

10. **Evaluate and Iterate**:
    Evaluate the quality of the generated poems based on various metrics like coherence, grammar, and overall poetic quality. Iterate on your model, data preprocessing, or hyperparameters to improve the results.

In [1]:
# importing required libraries
import os
import random
import nltk

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

import numpy as np 

import re
from nltk.tokenize import word_tokenize
import sentencepiece as spm

In [2]:
# saving list of names of txt files
directory_path = r'C:\Users\rohit\Python\Personal Projects\shaayar\ghazal_data\combined'

txt_files = [file for file in os.listdir(directory_path) if file.endswith('.txt')]

In [3]:
poems_data = ""

for txt_file in txt_files:
    file_path = os.path.join(directory_path, txt_file)
    
    with open(file_path, 'r') as file:
        poem_content = file.read()
    
    poems_data += poem_content

In [4]:
poems_data[:1000]

"tum apne shikve ki baten na khod khod ke puchho\nhazar karo mire dil se ki is men aag dabi hai\n\ndila ye dard-o-alam bhi to mughtanim hai ki akhir\nna girya-e-sahari hai na ah-e-nim-shabi hai\n\nki khar-e-khushk ko bhi dava-e-chaman-nasabi hai\nhua visal se shauq-e-dil-e-haris ziyada\n\nlab-e-qadah pe kaf-e-bada josh-e-tishna-labi hai\nkhusha vo dil ki sarapa tilism-e-be-khabari ho\n\njunun o yaas o alam rizq-e-muddaa-talabi hai\nchaman men kis ke ye barham hui hai bazm-e-tamasha\n\nki barg barg-e-saman shisha reza-e-halabi hai\n'ali' vali asadullah janashin-e-nabi hai\nhujum-e-nala hairat ajiz-e-arz yak-afghan hai\nkhamoshi resha-e-sad-nistan se khas-ba-dandan hai\n\nhui ye kasrat-e-gham se talaf kaifiyat-e-shadi\nki subh-e-id mujh ko bad-tar az-chak-e-gareban hai\n\ndil o diin naqd la saaqi se gar sauda kiya chahe\nki us bazar men saghar mata-e-dast-gardan hai\n\ngham aghosh-e-bala men parvarish deta hai ashiq ko\ncharagh-e-raushan apna qulzum-e-sarsar ka marjan hai\n\ntakalluf saz

In [5]:
len(poems_data)

915592

In [6]:
corpus = poems_data.lower().split('\n')
corpus = [line.replace('-', ' - ') + ' \n' for line in corpus]

In [7]:
corpus[:5]

['tum apne shikve ki baten na khod khod ke puchho \n',
 'hazar karo mire dil se ki is men aag dabi hai \n',
 ' \n',
 'dila ye dard - o - alam bhi to mughtanim hai ki akhir \n',
 'na girya - e - sahari hai na ah - e - nim - shabi hai \n']

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
total_words

10076

In [9]:
input_seqs = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram = token_list[:i+1]
        input_seqs.append(n_gram)

In [10]:
max_seq_len = max([len(x) for x in input_seqs])
max_seq_len

23

In [11]:
input_seqs = np.array(pad_sequences(input_seqs, maxlen = max_seq_len, padding = 'pre'))

In [12]:
X_train = input_seqs[:, :-1]
y_train = input_seqs[:, -1]

In [13]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes = total_words)

In [18]:
print(X_train.shape)
print(y_train.shape)

(167813, 22)
(167813, 10076)


In [19]:
model = Sequential()
model.add(Embedding(total_words, 240, input_length = max_seq_len - 1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation = 'softmax'))
adam = Adam(lr = 0.1)
model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs = 20, verbose = 1)

Epoch 1/20
5245/5245 [==============================] - 580s 110ms/step - loss: 6.4871 - accuracy: 0.0683
Epoch 2/20
5245/5245 [==============================] - 439s 84ms/step - loss: 5.7532 - accuracy: 0.1157
Epoch 3/20
5245/5245 [==============================] - 443s 84ms/step - loss: 5.2408 - accuracy: 0.1494
Epoch 4/20
 434/5245 [=>............................] - ETA: 6:46 - loss: 4.7630 - accuracy: 0.1830

KeyboardInterrupt: 

In [ ]:
seed_text = 'Uski aakhein jaise'
next_word = 20

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen = max_seq_len - 1, padding = 'pre')
    predicted = model.predict_classes(token_list, verbose = 0)
    output_word = ''
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += ' ' + output_word
print(seed_text)